# Linear Algebra

By now, we can load datasets into Arrays and manipulate these Arrays with basic mathematical operations. To start building sophisticated models, we will also need a few tools from linear algebra. This section offers a gentle introduction to the most essential concepts, starting from scalar arithmetic and ramping up to matrix multiplication.

## Scalars

Scalars are implemented as numbers. Below, we assign two scalars and perform the familiar addition, multiplication, division, and exponentiation operations.

In [1]:
x = 3.0
y = 2.0

x + y, x * y, x / y, x^y

(5.0, 6.0, 1.5, 9.0)

## Vectors

For our purposes, you can think of vectors as fixed-length arrays of scalars.

In [2]:
x = collect(1:3)

3-element Vector{Int64}:
 1
 2
 3

Recall that we access a vector’s elements via indexing.

In [3]:
x[3]

3

In code, this corresponds to the vector’s length, accessible via Julia’s built-in `length` function.

In [4]:
length(x)

3

We can also access the length via the `size` function. The size is a tuple that indicates a array’s length along each axis. Arrays with just one axis have shapes with just one element.

In [5]:
size(x)

(3,)

## Matrices

We can construct any appropriately sized `m x n` matrix by passing the desired shape to reshape:

In [6]:
A = reshape((1:6),3,2)

3×2 reshape(::UnitRange{Int64}, 3, 2) with eltype Int64:
 1  4
 2  5
 3  6

In code, we can access any matrix’s transpose as follows:

In [7]:
A'

2×3 adjoint(reshape(::UnitRange{Int64}, 3, 2)) with eltype Int64:
 1  2  3
 4  5  6

The following matrix is symmetric:

In [8]:
A = [1 2 3;2 0 4;3 4 5]
A == A'

true

Matrices are useful for representing datasets. Typically, rows correspond to individual records and columns correspond to distinct attributes.

## Arrays

Multi-dimensional arrays are constructed analogously to vectors and matrices, by growing the number of shape components.

In [9]:
reshape(1:24,2,3,4)

2×3×4 reshape(::UnitRange{Int64}, 2, 3, 4) with eltype Int64:
[:, :, 1] =
 1  3  5
 2  4  6

[:, :, 2] =
 7   9  11
 8  10  12

[:, :, 3] =
 13  15  17
 14  16  18

[:, :, 4] =
 19  21  23
 20  22  24

## Basic Properties of Array Arithmetic

Scalars, vectors, matrices, and Multi-dimensional arrays all have some handy properties. For example, elementwise operations produce outputs that have the same shape as their operands.

In [10]:
A = reshape(1:6,2,3)
B = copy(A)
display(A),display(A + B);

2×3 reshape(::UnitRange{Int64}, 2, 3) with eltype Int64:
 1  3  5
 2  4  6

2×3 Matrix{Int64}:
 2  6  10
 4  8  12

The elementwise product of two matrices is called their Hadamard product:

In [11]:
A .* B

2×3 Matrix{Int64}:
 1   9  25
 4  16  36

Adding or multiplying a scalar and an array produces a result with the same shape as the original array. Here, each element of the array is added to (or multiplied by) the scalar.

In [12]:
a = 2
X = reshape(1:24,2,3,4)
display(a .+ X)
size(a .* X)

2×3×4 Array{Int64, 3}:
[:, :, 1] =
 3  5  7
 4  6  8

[:, :, 2] =
  9  11  13
 10  12  14

[:, :, 3] =
 15  17  19
 16  18  20

[:, :, 4] =
 21  23  25
 22  24  26

(2, 3, 4)

## Reduction

Often, we wish to calculate the sum of a array’s elements.

In [13]:
x = Float32[1:3...]
x, sum(x)

(Float32[1.0, 2.0, 3.0], 6.0f0)

To express sums over the elements of arrays of arbitrary shape, we simply sum over all of its dimensions.

In [14]:
size(A),sum(A)

((2, 3), 21)

By default, invoking the `sum` function reduces a array length along all of its dimensions, but it will not producing a scalar. Julia also allow us to specify the dimensions along which the array should be reduced. To sum over all elements along the rows (dimension 2), we specify `dims=2` in `sum`. Since the input matrix reduces along dimension 2 to generate the output `2 x 1` matrix, this dimension length reduceds to 1. We can use `dropdims` to drop singleton dimensions in array.

In [15]:
size(A), size(sum(A,dims=2)),size(dropdims(sum(A,dims=2),dims=2))

((2, 3), (2, 1), (2,))

Specifying `dims=1` in `sum` funcion will generate a `1 x 3` matrix. 

In [16]:
size(A), size(sum(A,dims=1)),size(dropdims(sum(A,dims=1),dims=1))

((2, 3), (1, 3), (3,))

Reducing a matrix along both rows and columns via summation is not equivalent to summing up all the elements of the matrix, although matrix shape will reduce to (1,1) and has only one value 21.

In [17]:
first(sum(A,dims=(1,2))) == sum(A)

true

A related quantity is the `mean`, also called the average. We calculate the mean by dividing the sum by the total number of elements. Because computing the mean is so common, it gets a dedicated library function that works analogously to sum.

In [18]:
using Statistics
mean(A),sum(A)/length(A)

(3.5, 3.5)

Likewise, the function for calculating the mean can also reduce a array along specific dimensions.

In [19]:
mean(A,dims=2),sum(A,dims=2)/size(A,2)

([3.0; 4.0;;], [3.0; 4.0;;])